In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from IPython.display import Latex
from qiskit.visualization import state_visualization

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer

## The fusion for two trees at the root & the measurement-result-dependent local correction

### The root of left tree qubit (0) + the right tree qubit (1,2,3)

#### Y measurement at qubit 1: result is 1

In [50]:
prefix = "$$\\begin{align}"
suffix = "\\end{align}$$"
q = QuantumRegister(4)
circ = QuantumCircuit(q)

# Add a H gate on qubit 0, putting this qubit in superposition
circ.h(0)
circ.h(1)
circ.h(2)
circ.h(3)

# connect right subtree
circ.cz(1, 2)
circ.cz(1, 3)

circ.barrier()

# local complementation
circ.rx(np.pi/2, 1)
circ.rz(-np.pi/2, 2)
circ.rz(-np.pi/2, 3)

# connect the fusion node
circ.cz(0, 1)

# Y measurement on qubit 1
circ.sdg(1)
circ.h(1)

circ.barrier()

# local correction
# if y measurement result is 1
circ.s(0)
circ.s(2)
circ.s(3)

# if y measurement result is 0
# circ.p(-np.pi/2, 0)
# circ.p(-np.pi/2, 2)
# circ.p(-np.pi/2, 3)

# Set the initial state of the simulator to the ground state using from_int, then evolve the state by the quantum circuit
state = Statevector.from_int(0, 2**4)
state = state.evolve(circ)
state_to_latex = state_visualization._state_to_latex_ket(state.data, max_size = 128)
Latex(prefix + state_to_latex + suffix)

<IPython.core.display.Latex object>

In [51]:
circ.draw()

┌───┐       ░                             ░ ┌───┐
q32_0: ┤ H ├───────░──────────────■──────────────░─┤ S ├
       ├───┤       ░ ┌─────────┐  │ ┌─────┐┌───┐ ░ └───┘
q32_1: ┤ H ├─■──■──░─┤ Rx(π/2) ├──■─┤ Sdg ├┤ H ├─░──────
       ├───┤ │  │  ░ ├─────────┴┐   └─────┘└───┘ ░ ┌───┐
q32_2: ┤ H ├─■──┼──░─┤ Rz(-π/2) ├────────────────░─┤ S ├
       ├───┤    │  ░ ├──────────┤                ░ ├───┤
q32_3: ┤ H ├────■──░─┤ Rz(-π/2) ├────────────────░─┤ S ├
       └───┘       ░ └──────────┘                ░ └───┘

#### Y measurement at qubit 1: result is 0

In [52]:
prefix = "$$\\begin{align}"
suffix = "\\end{align}$$"
q = QuantumRegister(4)
circ = QuantumCircuit(q)

# Add a H gate on qubit 0, putting this qubit in superposition
circ.h(0)
circ.h(1)
circ.h(2)
circ.h(3)

# connect right subtree
circ.cz(1, 2)
circ.cz(1, 3)

circ.barrier()

# local complementation
circ.rx(np.pi/2, 1)
circ.rz(-np.pi/2, 2)
circ.rz(-np.pi/2, 3)

# connect the fusion node
circ.cz(0, 1)

# Y measurement on qubit 1
circ.sdg(1)
circ.h(1)

circ.barrier()

# local correction
# if y measurement result is 1
# circ.s(0)
# circ.s(2)
# circ.s(3)

# if y measurement result is 0
circ.p(-np.pi/2, 0)
circ.p(-np.pi/2, 2)
circ.p(-np.pi/2, 3)

# Set the initial state of the simulator to the ground state using from_int, then evolve the state by the quantum circuit
state = Statevector.from_int(0, 2**4)
state = state.evolve(circ)
state_to_latex = state_visualization._state_to_latex_ket(state.data, max_size = 128)
Latex(prefix + state_to_latex + suffix)

<IPython.core.display.Latex object>

In [53]:
circ.draw()

┌───┐       ░                             ░ ┌─────────┐
q33_0: ┤ H ├───────░──────────────■──────────────░─┤ P(-π/2) ├
       ├───┤       ░ ┌─────────┐  │ ┌─────┐┌───┐ ░ └─────────┘
q33_1: ┤ H ├─■──■──░─┤ Rx(π/2) ├──■─┤ Sdg ├┤ H ├─░────────────
       ├───┤ │  │  ░ ├─────────┴┐   └─────┘└───┘ ░ ┌─────────┐
q33_2: ┤ H ├─■──┼──░─┤ Rz(-π/2) ├────────────────░─┤ P(-π/2) ├
       ├───┤    │  ░ ├──────────┤                ░ ├─────────┤
q33_3: ┤ H ├────■──░─┤ Rz(-π/2) ├────────────────░─┤ P(-π/2) ├
       └───┘       ░ └──────────┘                ░ └─────────┘